# Query by Donor Contribution Context #

## Overview ##

Query based on `ctx_donor_contrib` view

## Notebook Setup ##

* Configure database connect information and options
* Clear potentially interfering context (PostgreSQL doesn't let you replace a view definition with conflicting column names)
* Set styling for notebook

In [1]:
sqlconnect = "postgresql+psycopg2://crash@localhost/fecdb"

%load_ext sql
%config SqlMagic.autopandas=True
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%sql $sqlconnect

'Connected: crash@fecdb'

In [2]:
%%html
<style>
  tr, th, td {
    text-align: left !important;
  }
</style>

## Validate Context ##

In [3]:
%%sql
select count(*)
  from ctx_donor_contrib

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,count
0,20567


## Query Based on Context ##

### Query using `ctx_donor_contrib` ###

In [4]:
%%sql
select dcx.elect_cycle,
       count(*) cycle_contribs,
       sum(dcx.transaction_amt) cycle_amount,
       round(avg(dcx.transaction_amt), 2) avg_amount,
       min(dcx.transaction_amt) min_amount,
       max(dcx.transaction_amt) max_amount
  from ctx_donor_contrib dcx
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
11 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2000,646,6882505.00,10654.03,-245000.00,317068.00
1,2002,708,32106740.00,45348.50,-1000.00,7000000.00
2,2004,799,28080591.00,35144.67,-2000.00,3000000.00
3,2006,922,33215996.00,36026.03,-1000.00,2500000.00
4,2008,1015,15334856.00,15108.23,-5000.00,2878872.00
5,2010,1323,105298333.00,79590.58,-100000.00,6000000.00
6,2012,1529,291771883.00,190825.30,-750000.00,10000000.00
7,2014,2341,201218838.00,85954.22,-10200.00,16000000.00
8,2016,4143,722705756.00,174440.20,-226800.00,11000000.00
9,2018,6033,665768670.00,110354.50,-54000.00,20000000.00


In [5]:
%%sql
select d.id as donor_id,
       d.name as donor_name,
       count(*) contribs,
       sum(dcx.transaction_amt) total_amount,
       round(avg(dcx.transaction_amt), 2) avg_amount,
       max(dcx.transaction_amt) max_amount,
       array_agg(distinct dcx.elect_cycle) as elect_cycles,
       round(sum(dcx.transaction_amt) / count(distinct dcx.elect_cycle), 2) avg_cycle_amount
  from ctx_donor_contrib dcx
  join donor_indiv d on d.id = dcx.donor_indiv_id
 group by 1, 2
 order by 4 desc
 limit 50

 * postgresql+psycopg2://crash@localhost/fecdb
50 rows affected.


,donor_id,donor_name,contribs,total_amount,avg_amount,max_amount,elect_cycles,avg_cycle_amount
0,11612429,"STEYER, THOMAS",288,259466083.00,900923.90,16000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",23587825.73
1,70790,"ADELSON, SHELDON",521,163010720.00,312880.46,15000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",14819156.36
2,1076630,"BLOOMBERG, MICHAEL",368,162000273.00,440218.13,20000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",14727297.55
3,70692,"ADELSON, MIRIAM",449,148007082.00,329637.15,15000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",13455189.27
4,3535137,"EYCHANER, FRED",602,80924450.00,134426.00,5000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",7356768.18
5,7961557,"MCMAHON, LINDA",197,75964481.00,385606.50,6000000.00,"[2004, 2006, 2010, 2012, 2014, 2016, 2020]",10852068.71
6,12286548,"UIHLEIN, RICHARD",1111,73472265.00,66131.65,3500000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",6679296.82
7,10804004,"SCOTT, RICK",169,69942510.00,413861.01,7450000.00,[2018],69942510.00
8,11121369,"SIMONS, JAMES",404,54242910.00,134264.63,4000000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",4931173.64
9,11141923,"SINGER, PAUL",823,53184692.00,64622.96,2500000.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201...",4834972.00


In [6]:
%%sql
select d.id as donor_id,
       d.name as donor_name,
       dcx.elect_cycle,
       count(*) cycle_contribs,
       sum(dcx.transaction_amt) cycle_total_amount,
       round(avg(dcx.transaction_amt), 2) cycle_avg_amount,
       max(dcx.transaction_amt) cycle_max_amount
  from ctx_donor_contrib dcx
  join donor_indiv d on d.id = dcx.donor_indiv_id
 group by 1, 2, 3
 order by 5 desc
 limit 50

 * postgresql+psycopg2://crash@localhost/fecdb
50 rows affected.


,donor_id,donor_name,elect_cycle,cycle_contribs,cycle_total_amount,cycle_avg_amount,cycle_max_amount
0,1076630,"BLOOMBERG, MICHAEL",2018,56,95110857.00,1698408.16,20000000.00
1,11612429,"STEYER, THOMAS",2016,69,89988044.00,1304174.55,11000000.00
2,11612429,"STEYER, THOMAS",2014,27,78845867.00,2920217.30,16000000.00
3,11612429,"STEYER, THOMAS",2018,59,73089512.00,1238805.29,13504144.00
4,10804004,"SCOTT, RICK",2018,169,69942510.00,413861.01,7450000.00
5,70790,"ADELSON, SHELDON",2018,117,61971800.00,529673.50,15000000.00
6,70692,"ADELSON, MIRIAM",2018,117,61757400.00,527841.03,15000000.00
7,7961557,"MCMAHON, LINDA",2010,144,53107350.00,368801.04,6000000.00
8,70790,"ADELSON, SHELDON",2012,61,51023525.00,836451.23,5000000.00
9,70692,"ADELSON, MIRIAM",2012,50,46626900.00,932538.00,5000000.00
